In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfo = pd.read_sql_table('race', conn)
print(f'{len(dfo)} races loaded!')

9299 races loaded!


In [3]:
category = 'G'
dfo = dfo.loc[dfo['race_type'].isin([category])]
print(f'{len(dfo)} {category} races!')

3796 G races!


In [4]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
0,1,RICHMOND,NSW,C,G,2017-09-13,1,LADBROKES INFO HUB MDN HT1,2017-09-13 05:12:00,Paying,400,"[[4], [2], [1], [5]]",NaN,"[{""runnerName"": ""ZALI'S BLUEBOY"", ""runnerNumbe...","[[4], [2], [1], [5]]","[{'runnerName': 'ZALI'S BLUEBOY', 'runnerNumbe..."
1,2,RICHMOND,NSW,C,G,2017-09-13,2,KEVIN WATERS TOWING MDN HT2,2017-09-13 05:32:00,Paying,400,"[[1], [8], [7], [2]]",NaN,"[{""runnerName"": ""SKETCHY LINDA"", ""runnerNumber...","[[1], [8], [7], [2]]","[{'runnerName': 'SKETCHY LINDA', 'runnerNumber..."
2,3,RICHMOND,NSW,C,G,2017-09-13,4,PENRITHELECTRICAL.COM.AU STKS,2017-09-13 06:07:00,Paying,400,"[[9], [7], [8], [2]]",NaN,"[{""runnerName"": ""BROWZA BOB'"", ""runnerNumber"":...","[[9], [7], [8], [2]]","[{'runnerName': 'BROWZA BOB'', 'runnerNumber':..."
3,4,RICHMOND,NSW,C,G,2017-09-13,5,RICHMONDGREYHOUNDS.COM.AU STKS,2017-09-13 06:27:00,Paying,618,"[[6], [7], [3], [1]]",NaN,"[{""runnerName"": ""MIAMI DEMON"", ""runnerNumber"":...","[[6], [7], [3], [1]]","[{'runnerName': 'MIAMI DEMON', 'runnerNumber':..."
4,5,RICHMOND,NSW,C,G,2017-09-13,6,SWIFT FANCY @ STUD 1-2W STKS,2017-09-13 06:42:00,Paying,535,"[[6], [7], [1], [8]]",NaN,"[{""runnerName"": ""KILLING TIME"", ""runnerNumber""...","[[6], [7], [1], [8]]","[{'runnerName': 'KILLING TIME', 'runnerNumber'..."


In [13]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['odds_win']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,odds_fperc,odds_fscale,odds_fwin,odds_perc,odds_scale,...,odds_win,parimutuel,race_type,rank_fwin,rank_twin,rank_win,riderDriverName,runnerName,runnerNumber,trainerName
0,0,0,0,"{'returnWin': 0, 'returnWinOpen': 0, 'returnPl...",8,NaN,NaN,NaN,0.000000,0.000000,...,0.00,"{'bettingStatus': 'Scratched', 'returnWin': 0,...",G,NaN,NaN,10,,STYLE,9,JUSTIN WOOD
1,7,0,1,"{'returnWin': 16, 'returnWinOpen': 11, 'return...",8,NaN,NaN,NaN,0.062500,0.051328,...,16.00,"{'bettingStatus': 'Normal', 'returnWin': 12.3,...",G,NaN,NaN,5,,SLEEPY AGAIN,10,GRAEME HALL
2,3,0,3,"{'returnWin': 16, 'returnWinOpen': 15, 'return...",8,NaN,NaN,NaN,0.062500,0.051328,...,16.00,"{'bettingStatus': 'Normal', 'returnWin': 14.5,...",G,NaN,NaN,5,,LIL MISS CHEEKY,3,GRAEME HALL
3,4,0,0,"{'returnWin': 16, 'returnWinOpen': 8.5, 'retur...",8,NaN,NaN,NaN,0.062500,0.051328,...,16.00,"{'bettingStatus': 'Normal', 'returnWin': 20.4,...",G,NaN,NaN,5,,KING DEBELLO,4,DEREK COLDSTREAM
4,5,0,0,"{'returnWin': 26, 'returnWinOpen': 17, 'return...",8,NaN,NaN,NaN,0.038462,0.031586,...,26.00,"{'bettingStatus': 'Normal', 'returnWin': 32.8,...",G,NaN,NaN,8,,MISSONI,5,MAY LEACH
5,6,0,0,"{'returnWin': 3.7, 'returnWinOpen': 6, 'return...",8,NaN,NaN,NaN,0.270270,0.221959,...,3.70,"{'bettingStatus': 'Normal', 'returnWin': 4, 'r...",G,NaN,NaN,2,,WHO'S GOT STORM,6,TONY SCOTT
6,7,0,0,"{'returnWin': 0, 'returnWinOpen': 0, 'returnPl...",8,NaN,NaN,NaN,0.000000,0.000000,...,0.00,"{'bettingStatus': 'Scratched', 'returnWin': 0,...",G,NaN,NaN,10,,GOING TO JACKSON,7,BEN MACLEAN
7,8,0,2,"{'returnWin': 15, 'returnWinOpen': 8.5, 'retur...",8,NaN,NaN,NaN,0.066667,0.054750,...,15.00,"{'bettingStatus': 'Normal', 'returnWin': 8.5, ...",G,NaN,NaN,4,,NEW COURSE,8,JOHN RIGHETTI
8,1,0,0,"{'returnWin': 1.75, 'returnWinOpen': 1.65, 're...",8,NaN,NaN,NaN,0.571429,0.469284,...,1.75,"{'bettingStatus': 'Normal', 'returnWin': 1.9, ...",G,NaN,NaN,1,,JUNKYARD SARGE,1,JAMES JEFFRIES
9,2,0,4,"{'returnWin': 12, 'returnWinOpen': 11, 'return...",8,NaN,NaN,NaN,0.083333,0.068437,...,12.00,"{'bettingStatus': 'Normal', 'returnWin': 13.6,...",G,NaN,NaN,3,,HARLAN ROCKS,2,KODY CHARLES


In [14]:
# drop scratched
data = data_all.dropna(subset=['odds_win'])
data = data[(data.odds_win > 0)]
data.describe()

,barrierNumber,claimAmount,finishingPosition,num_runners,odds_fperc,odds_fscale,odds_fwin,odds_perc,odds_scale,odds_tperc,odds_tscale,odds_twin,odds_win,rank_fwin,rank_twin,rank_win,runnerNumber
count,29006.000000,29006.0,29006.000000,29006.000000,11515.000000,11515.000000,11515.000000,29006.000000,29006.000000,11515.000000,11515.000000,11515.000000,29006.000000,11515.000000,11515.000000,29006.000000,29006.000000
mean,4.393712,0.0,1.250052,7.720265,0.160287,0.130439,13.386566,0.163178,0.130869,0.154652,0.130439,13.801931,13.207495,4.360313,4.395745,4.276805,4.608150
std,2.273991,0.0,1.458515,0.728711,0.138495,0.111572,14.008361,0.140159,0.109446,0.137515,0.115668,13.917178,13.605410,2.321031,2.352761,2.227413,2.433648
min,0.000000,0.0,0.000000,4.000000,0.000000,0.000000,1.010000,0.006623,0.003072,0.000000,0.000000,0.000000,1.010000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,0.0,0.000000,8.000000,0.055556,0.045869,4.400000,0.058824,0.046382,0.053191,0.044817,4.600000,4.400000,2.000000,2.000000,2.000000,3.000000
50%,4.000000,0.0,1.000000,8.000000,0.117647,0.097343,8.500000,0.125000,0.098345,0.111111,0.093917,8.700000,8.000000,4.000000,4.000000,4.000000,5.000000
75%,6.000000,0.0,2.000000,8.000000,0.227273,0.180584,18.000000,0.227273,0.181155,0.212766,0.179946,18.200000,17.000000,6.000000,6.000000,6.000000,7.000000
max,8.000000,0.0,4.000000,10.000000,0.990099,0.754156,151.000000,0.990099,0.774723,0.961538,0.787844,168.500000,151.000000,11.000000,12.000000,10.000000,10.000000


In [15]:
# get label data
Y = data['finishingPosition'] == 1
Y = Y.astype(int)
Y.describe()

count    29006.000000
mean         0.131076
std          0.337490
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [16]:
# xp = fixed odds perc
xp = data['odds_perc']
xp.describe()

count    29006.000000
mean         0.163178
std          0.140159
min          0.006623
25%          0.058824
50%          0.125000
75%          0.227273
max          0.990099
Name: odds_perc, dtype: float64

In [17]:
# xs = odds scaled
xs = data['odds_scale']
xs.describe()

count    29006.000000
mean         0.130869
std          0.109446
min          0.003072
25%          0.046382
50%          0.098345
75%          0.181155
max          0.774723
Name: odds_scale, dtype: float64

In [18]:
# xr = win rank
xr = data['rank_win']
xr.describe()

count    29006.000000
mean         4.276805
std          2.227413
min          1.000000
25%          2.000000
50%          4.000000
75%          6.000000
max         10.000000
Name: rank_win, dtype: float64

In [19]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    29006.000000
mean         7.720265
std          0.728711
min          4.000000
25%          8.000000
50%          8.000000
75%          8.000000
max         10.000000
Name: num_runners, dtype: float64

In [20]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,odds_perc,odds_scale,rank_win,num_runners
count,29006.000000,29006.000000,29006.000000,29006.000000
mean,0.163178,0.130869,4.276805,7.720265
std,0.140159,0.109446,2.227413,0.728711
min,0.006623,0.003072,1.000000,4.000000
25%,0.058824,0.046382,2.000000,8.000000
50%,0.125000,0.098345,4.000000,8.000000
75%,0.227273,0.181155,6.000000,8.000000
max,0.990099,0.774723,10.000000,10.000000


In [21]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30'.format(category)
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 1000


In [24]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)